# Flask #

## Background ##

### Every web server must have a unique IP address###
![title](img/ip.png)
#### Special IP addresses ####
**Loopback (self reference) **
127.0.0.1
**Private IP Addresses**
10.0.0.0
172.16.0.0
192.168.0.0

### Every web application must have a unique port number ###
HTTP is the language of the web using GET and POST and typically runs on port 80.

Try visiting www.google.com:80 on your browser, then change the port from 80 to something else and try again. What happened?

#### How HTTP works ####
![title](img/http.png)
- Web browser requests a web page from the web server using GET
- Web server responses with the web page with a POST

# Let's Code! #

1. Install and import flask
2. Build a basic hello world web page
3. Create an HTML form that can send data to the flask application and return answer in a new webpage

## Part 1: Installing and importing flask ##

In [1]:
# use `conda install flask`
# to install
import sys
from flask import Flask, render_template, request
import pandas as pd
app = Flask(__name__)

## Part 2: Basic hello world html page ##

In [2]:
@app.route("/hello") #basic hello world
def hello():
    return "Hello World!"

@app.route("/html") #a full html page
def hellohtml():
    return """
    <!DOCTYPE html>
<html>
<head>
<title>Hello World!</title>
</head>

<body>
Welcome to DAT2 Science!
</body>

</html>
"""

## Part 3: HTML form to send and receive data ##
Now we will reference a simple HTML form to send an output back to our python program which we can use for processing

In [3]:
#create a default route that supports both GET and POST methods
results = []
@app.route('/', methods=['GET', 'POST'])
def index():
    errors = []
#     results = []
    if request.method == "POST":
        # get url that the user has entered
        try:
            print(request.form)
            features={}
            features['Hamlet'] = request.form['hamlet']
            features['Container_ID'] = request.form['containertype']
            features['Water_level'] = request.form['waterlevel']
            if 'meso' in request.form:
                features['Meso']=1
            else:
                features['Meso']=0
            if 'containerpos' in request.form:
                features['Container_position']=1
            else:
                features['Container_position']=0

            if 'chemicals' in request.form:
                features['Treatment']=1
            else:
                features['Treatment']=0
        except:
            print sys.exc_info()[0]
            errors.append(
                "Could not get data from HTML form. Please try again?"
            )
            return render_template('index.html', errors=errors)
        
        x = validateInput(features)
        if x!= 0:
            errors.append(x)
            
        #convert dict to dataframe for analysis
        y=[features]
        df=pd.DataFrame(y)
        #re-order to ensure dict conversion was good
        df = df[['Hamlet', 'Container_ID', 'Waterlevel', 'Container_position','Meso','Treatment']]
        
        ans = predict(df)
        
        hamlet=getHamlet(features['Hamlet'])
        container=getContainer(features['Container_ID'])
        if ans[0]==True:
            larvae="Yes"
        else:
            larvae="No"
        results.append([hamlet,container,larvae,str(ans[1])])
    return render_template('index.html', errors=errors, results=results)

def validateInput(features):
    try:
        features['Hamlet']=int(features['Hamlet'])
        features['Container_ID']=int(features['Container_ID'])
        features['Water_level']=int(features['Water_level'])
        features['Meso']=int(features['Meso'])
        features['Container_position']=int(features['Container_position'])
        features['Treatment']=int(features['Treatment'])
    except:
        return "Invalid input, please try again?"
    
    return 0

def getHamlet(hamletnum):
    hamlets = ["Tran An","Tan My","My Hanh","My Chanh","My Doi","Hoa Phu","Rach Doi","Ao Cho","Luang Khanh","Quang Hao","Tan Hanh"]
    if hamletnum >= 0 and hamletnum <=10:
        return hamlets[hamletnum]
    return ""

def getContainer(containernum):
    containers = ["Box Tank","Cylindrical Tank","Standard Jar","Small Jar","Large Jar","Other"]
    if containernum >= 1 and containernum <=6:
        return containers[containernum-1]
    return ""

def predict(features):
    return [True,25]

## Run cell below to start web server ##

In [ ]:
if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jan/2017 03:53:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2017 03:53:55] "POST / HTTP/1.1" 200 -


ImmutableMultiDict([('containertype', u'5'), ('containerpos', u'1'), ('hamlet', u'8'), ('meso', u'1'), ('waterlevel', u'5')])
